In [1]:
import os
import pandas as pd
import geopandas as gpd
from IPython.display import display
import numpy as np

In [2]:
# read and check datasets in the raw file
external = pd.DataFrame()
path = os.getcwd().replace("notebooks","") + "data/raw/"
os.listdir(path)

['income.xlsx',
 'shapefile.zip',
 '.DS_Store',
 'covid_test.csv',
 'cemetery.csv',
 'shapefiles',
 'Median_Suburb_Quarterly_2020.xls',
 'Median_Suburb_Time_2021.xls',
 'Rental_Report_Quarterly_Median_2020.xlsx',
 '.gitkeep',
 'school_2021.csv',
 'functional.xlsx',
 'Rental_Report_Quarterly_Median_2021.xlsx',
 'school_2020.csv',
 'school_2022.csv',
 'Affordable_Lettings_2020.xlsx',
 'Gaming_Expenditure.xls',
 'Population_density_gaming_expenditures.xls',
 'datalink.txt',
 'Rental_Report_Quarterly.xlsx',
 'Government_Organisations.xlsx',
 'GNR.csv',
 'Sport_Recreational_Facilities_list.xlsx',
 'Affordable_Lettings_2021.xlsx',
 'Median_Suburb_Quarterly_2021.xlsx',
 'australian_postcodes.csv',
 'property.json',
 'ambulance.csv',
 'crime.xlsx',
 'population.xlsx']

In [3]:
# read Australian_postcodes
postcode = pd.read_csv("../data/raw/australian_postcodes.csv")
postcode.columns

Index(['id', 'postcode', 'locality', 'state', 'long', 'lat', 'dc', 'type',
       'status', 'sa3', 'sa3name', 'sa4', 'sa4name', 'region', 'Lat_precise',
       'Long_precise', 'SA1_MAINCODE_2011', 'SA1_MAINCODE_2016',
       'SA2_MAINCODE_2016', 'SA2_NAME_2016', 'SA3_CODE_2016', 'SA3_NAME_2016',
       'SA4_CODE_2016', 'SA4_NAME_2016', 'RA_2011', 'RA_2016', 'MMM_2015',
       'MMM_2019', 'ced', 'altitude', 'chargezone', 'phn_code', 'phn_name',
       'lgaregion', 'electorate', 'electoraterating'],
      dtype='object')

In [4]:
# extract postcode, SA2_MAINCODE_2016, SA2_NAME_2016 from postcode to external
external['postcode'] = postcode['postcode']
external['locality'] = postcode['locality']
external['state'] = postcode['state']
external['LGA'] = postcode['lgaregion']
external['SA3_NAME_2016'] = postcode['SA3_NAME_2016']
external['SA2_Code'] = postcode['SA2_MAINCODE_2016']
external['SA2_Name'] = postcode['SA2_NAME_2016']
external = external[external['state'] == "VIC"]
external.reset_index(drop=True, inplace=True)
external

,postcode,locality,state,LGA,SA3_NAME_2016,SA2_Code,SA2_Name
0,3000,MELBOURNE,VIC,Melbourne,Melbourne City,206041122.0,Melbourne
1,3001,MELBOURNE,VIC,Moonee Valley,Melbourne City,206041122.0,Melbourne
2,3002,EAST MELBOURNE,VIC,Yarra,Melbourne City,206041119.0,East Melbourne
3,3003,WEST MELBOURNE,VIC,Melbourne,Melbourne City,206041127.0,West Melbourne
4,3004,MELBOURNE,VIC,Yarra,Melbourne City,206041126.0,Southbank
...,...,...,...,...,...,...,...
3529,8111,MELBOURNE,VIC,Yarra,Melbourne City,206041122.0,Melbourne
3530,8120,MELBOURNE,VIC,Yarra,Melbourne City,206041122.0,Melbourne
3531,8205,MELBOURNE,VIC,Yarra,Melbourne City,206041122.0,Melbourne
3532,8785,DANDENONG,VIC,Monash,Dandenong,212041311.0,Dandenong


In [5]:
# read data, Table 1 in population
population = pd.read_excel("../data/raw/population.xlsx",sheet_name="Table 1",header=7)
#population.columns
population = population.drop(columns=['SA2 name','S/T code', 'S/T name', 'GCCSA code', 'GCCSA name', 'SA4 code','SA4 name',
                                      'SA3 code', 'SA3 name',"Unnamed: 31","Unnamed: 34"])
population.head(5)

,SA2 code,no.,no..1,no..2,no..3,no..4,no..5,no..6,no..7,no..8,...,no..15,no..16,no..17,no..18,no..19,no..20,no..21,%,km2,persons/km2
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,101021007.0,2760.0,2811.0,2835.0,2844.0,2847.0,2965.0,3102.0,3181.0,3308.0,...,3950.0,4039.0,4140.0,4211.0,4273.0,4330.0,872.0,25.2,3418.4,1.3
2,101021008.0,9129.0,9199.0,9263.0,9277.0,9209.0,9212.0,9033.0,8994.0,9030.0,...,8531.0,8526.0,8507.0,8488.0,8519.0,8546.0,-546.0,-6.0,7.0,1223.9
3,101021009.0,9717.0,9513.0,9522.0,9400.0,9595.0,9682.0,9793.0,10074.0,10288.0,...,11230.0,11355.0,11447.0,11450.0,11437.0,11370.0,655.0,6.1,4.8,2387.7
4,101021010.0,3925.0,4073.0,4219.0,4218.0,4187.0,4319.0,4459.0,4595.0,4712.0,...,4970.0,5013.0,5072.0,5117.0,5077.0,5093.0,142.0,2.9,13.0,391.7


In [6]:
# change the column name
population.columns = ['SA2_Code','2001_population','2002_population','2003_population',
                      '2004_population','2005_population','2006_population', '2007_population','2008_population',
                      '2009_population', '2010_population', '2011_population', '2012_population', '2013_population',
                      '2014_population', '2015_population', '2016_population', '2017_population','2018_population',
                      '2019_population', '2020_population', '2021_population',"Population_Change_no",
                      "Population_Change_percent","Population_Area_km2","Population_density_2021_persons/km2"]

# remove the null columns
population = population.dropna()
display(population.tail(5))
population.shape

,SA2_Code,2001_population,2002_population,2003_population,2004_population,2005_population,2006_population,2007_population,2008_population,2009_population,...,2016_population,2017_population,2018_population,2019_population,2020_population,2021_population,Population_Change_no,Population_Change_percent,Population_Area_km2,Population_density_2021_persons/km2
2450,801111141.0,12.0,11.0,11.0,10.0,10.0,9.0,15.0,22.0,27.0,...,39.0,45.0,50.0,56.0,61.0,67.0,26.0,63.4,1202.8,0.1
2451,901011001.0,1442.0,1365.0,1337.0,1355.0,1380.0,1403.0,1569.0,1745.0,1944.0,...,1903.0,1877.0,1849.0,1801.0,1752.0,1716.0,-451.0,-20.8,136.1,12.6
2452,901021002.0,600.0,568.0,558.0,573.0,588.0,590.0,575.0,568.0,565.0,...,546.0,569.0,571.0,599.0,605.0,602.0,41.0,7.3,13.7,43.9
2453,901031003.0,542.0,464.0,441.0,428.0,413.0,386.0,370.0,370.0,367.0,...,402.0,392.0,376.0,360.0,338.0,310.0,-79.0,-20.3,67.2,4.6
2454,901041004.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1757.0,1827.0,1920.0,1999.0,2096.0,2220.0,2220.0,0.0,38.7,57.4


(2454, 26)

In [7]:
# merge the external and population dataset together, fill null with NaN
external = pd.merge(external, population, on='SA2_Code', how='left').fillna(np.nan)
external

,postcode,locality,state,LGA,SA3_NAME_2016,SA2_Code,SA2_Name,2001_population,2002_population,2003_population,...,2016_population,2017_population,2018_population,2019_population,2020_population,2021_population,Population_Change_no,Population_Change_percent,Population_Area_km2,Population_density_2021_persons/km2
0,3000,MELBOURNE,VIC,Melbourne,Melbourne City,206041122.0,Melbourne,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3001,MELBOURNE,VIC,Moonee Valley,Melbourne City,206041122.0,Melbourne,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3002,EAST MELBOURNE,VIC,Yarra,Melbourne City,206041119.0,East Melbourne,3731.0,3859.0,4243.0,...,5475.0,5495.0,5409.0,5413.0,5378.0,4962.0,-276.0,-5.3,2.9,1711.2
3,3003,WEST MELBOURNE,VIC,Melbourne,Melbourne City,206041127.0,West Melbourne,0.0,0.0,0.0,...,3.0,3.0,3.0,3.0,0.0,0.0,0.0,0.0,6.2,0.0
4,3004,MELBOURNE,VIC,Yarra,Melbourne City,206041126.0,Southbank,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3529,8111,MELBOURNE,VIC,Yarra,Melbourne City,206041122.0,Melbourne,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3530,8120,MELBOURNE,VIC,Yarra,Melbourne City,206041122.0,Melbourne,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3531,8205,MELBOURNE,VIC,Yarra,Melbourne City,206041122.0,Melbourne,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3532,8785,DANDENONG,VIC,Monash,Dandenong,212041311.0,Dandenong,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
# read data, Table 1.4 in income
income = pd.read_excel("../data/raw/income.xlsx",sheet_name="Table 1.4",header=6)
display(income.head(2))
# extract the colunms of year
income = income.drop(columns=['SA2 NAME','2014-15', '2015-16', '2016-17', '2017-18','2018-19', '2014-15.1', 
                              '2015-16.1', '2016-17.1', '2017-18.1','2018-19.1', '2014-15.2', '2015-16.2', 
                              '2016-17.2', '2017-18.2','2018-19.2','Unnamed: 27'])
income.columns = ['SA2_Code','2014-15_median_income','2015-16_median_income','2016-17_median_income',
                  '2017-18_median_income','2018-19_median_income','2014-15_mean_income','2015-16_mean_income',
                  '2016-17_mean_income','2017-18_mean_income','2018-19_mean_income']

# replace np with NaN
income = income.replace('np', np.nan)
income.head(3)

,SA2,SA2 NAME,2014-15,2015-16,2016-17,2017-18,2018-19,2014-15.1,2015-16.1,2016-17.1,...,2015-16.3,2016-17.3,2017-18.3,2018-19.3,2014-15.4,2015-16.4,2016-17.4,2017-18.4,2018-19.4,Unnamed: 27
0,Australia,NaN,13102895,13358252,13678024,14069082,14425037,42,42,42,...,47692,48360,49805,51389,61036,61975,62594,64246,65953,NaN
1,New South Wales,NaN,4091347,4191542,4344997,4466941,4569650,42,42,42,...,48085,48700,50153,51818,62798,64493,65196,67200,68816,NaN


,SA2_Code,2014-15_median_income,2015-16_median_income,2016-17_median_income,2017-18_median_income,2018-19_median_income,2014-15_mean_income,2015-16_mean_income,2016-17_mean_income,2017-18_mean_income,2018-19_mean_income
0,Australia,46854.0,47692.0,48360,49805.0,51389.0,61036.0,61975.0,62594.0,64246.0,65953.0
1,New South Wales,46879.0,48085.0,48700,50153.0,51818.0,62798.0,64493.0,65196.0,67200.0,68816.0
2,101021007,38093.0,39716.0,41288,42003.0,41593.0,47741.0,51074.0,51090.0,51594.0,51149.0


In [9]:
# merge the exrternal with income data
external = pd.merge(external, income, on='SA2_Code', how='left').fillna(np.nan)
external

,postcode,locality,state,LGA,SA3_NAME_2016,SA2_Code,SA2_Name,2001_population,2002_population,2003_population,...,2014-15_median_income,2015-16_median_income,2016-17_median_income,2017-18_median_income,2018-19_median_income,2014-15_mean_income,2015-16_mean_income,2016-17_mean_income,2017-18_mean_income,2018-19_mean_income
0,3000,MELBOURNE,VIC,Melbourne,Melbourne City,206041122.0,Melbourne,NaN,NaN,NaN,...,32697.0,31242.0,28068.0,28806.0,30024.0,53183.0,53292.0,51261.0,51528.0,52476.0
1,3001,MELBOURNE,VIC,Moonee Valley,Melbourne City,206041122.0,Melbourne,NaN,NaN,NaN,...,32697.0,31242.0,28068.0,28806.0,30024.0,53183.0,53292.0,51261.0,51528.0,52476.0
2,3002,EAST MELBOURNE,VIC,Yarra,Melbourne City,206041119.0,East Melbourne,3731.0,3859.0,4243.0,...,67966.0,71551.0,70034.0,71759.0,75755.0,114762.0,122960.0,124057.0,128069.0,132130.0
3,3003,WEST MELBOURNE,VIC,Melbourne,Melbourne City,206041127.0,West Melbourne,0.0,0.0,0.0,...,NaN,NaN,58023.0,93272.0,82813.0,NaN,NaN,107800.0,237158.0,112680.0
4,3004,MELBOURNE,VIC,Yarra,Melbourne City,206041126.0,Southbank,NaN,NaN,NaN,...,54176.0,54683.0,50643.0,51696.0,53390.0,73998.0,75285.0,72915.0,73634.0,75071.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3529,8111,MELBOURNE,VIC,Yarra,Melbourne City,206041122.0,Melbourne,NaN,NaN,NaN,...,32697.0,31242.0,28068.0,28806.0,30024.0,53183.0,53292.0,51261.0,51528.0,52476.0
3530,8120,MELBOURNE,VIC,Yarra,Melbourne City,206041122.0,Melbourne,NaN,NaN,NaN,...,32697.0,31242.0,28068.0,28806.0,30024.0,53183.0,53292.0,51261.0,51528.0,52476.0
3531,8205,MELBOURNE,VIC,Yarra,Melbourne City,206041122.0,Melbourne,NaN,NaN,NaN,...,32697.0,31242.0,28068.0,28806.0,30024.0,53183.0,53292.0,51261.0,51528.0,52476.0
3532,8785,DANDENONG,VIC,Monash,Dandenong,212041311.0,Dandenong,NaN,NaN,NaN,...,37000.0,36587.0,37323.0,39276.0,40965.0,41083.0,41195.0,42502.0,44205.0,45908.0


In [10]:
# helper function for count the total area by SA2 Area
def sum_area(df):
    df["total"] = 0
    col = df.columns
    df["total"] = df[col[1]]+df[col[2]]+df[col[3]]+df[col[4]]+df[col[5]]+df[col[6]]+df[col[7]]
    return df

In [11]:
# read Land and Housing Supply Indicators data and extract the data we needed
residential_area = pd.read_excel("../data/raw/functional.xlsx",sheet_name="Table 1",header=5)
residential_area.columns = ['SA2_Code', 'SA2 name', "residential_area<200sqm","residential_area_200-400sqm",
                           "residential_area_400-600sqm","residential_area_600-800sqm","residential_area_800-1000sqm",
                           "residential_area_1000-10000sqm","residential_area>10000sqm"]
residential_area = residential_area.drop(columns=['SA2 name'])
sum_area(residential_area)

primary_production_area = pd.read_excel("../data/raw/functional.xlsx",sheet_name="Table 3",header=5)
primary_production_area.columns = ['SA2_Code', 'SA2 name', "primary_production<200sqm","primary_production_200-400sqm",
                           "primary_production_400-600sqm","primary_production_600-800sqm",
                           "primary_production_800-1000sqm","primary_production_1000-10000sqm",
                           "primary_production>10000sqm"]
primary_production_area = primary_production_area.drop(columns=['SA2 name'])
sum_area(primary_production_area)

community_infrastructure_area = pd.read_excel("../data/raw/functional.xlsx",sheet_name="Table 3",header=5)
community_infrastructure_area.columns = ['SA2_Code', 'SA2 name', "community_infrastructure<200sqm",
                                         "community_infrastructure_200-400sqm","community_infrastructure_400-600sqm",
                                         "community_infrastructure_600-800sqm","community_infrastructure_800-1000sqm",
                                         "community_infrastructure_1000-10000sqm","community_infrastructure>10000sqm"]
community_infrastructure_area = community_infrastructure_area.drop(columns=['SA2 name'])
sum_area(community_infrastructure_area)

mix_use_area = pd.read_excel("../data/raw/functional.xlsx",sheet_name="Table 5",header=5)
mix_use_area.columns = ['SA2_Code', 'SA2 name','mix_area<200sqm', 'mix_area_200-400sqm', 'mix_area_400-600sqm',
                        'mix_area_600-800sqm', 'mix_area_800-1000sqm', 'mix_area_1000-10000sqm', 'mix_area>1000sqm']
mix_use_area = mix_use_area.drop(columns=['SA2 name'])
sum_area(mix_use_area)

industrial_area = pd.read_excel("../data/raw/functional.xlsx",sheet_name="Table 6",header=5)
industrial_area.columns = ['SA2_Code', 'SA2 name','industrial_area<200sqm', 'industrial_area_200-400sqm',
                           'industrial_area_400-600sqm', 'industrial_area_600-800sqm', 'industrial_area_800-1000sqm',
                           'industrial_area_1000-10000sqm', 'industrial_area>1000sqm']
industrial_area = industrial_area.drop(columns=['SA2 name'])
sum_area(industrial_area)

scenic_spot_area = pd.read_excel("../data/raw/functional.xlsx",sheet_name="Table 7",header=5)
scenic_spot_area.columns = ['SA2_Code', 'SA2 name','scenic_spot<200sqm', 'scenic_spot_200-400sqm',
                             'scenic_spot_400-600sqm', 'scenic_spot_600-800sqm', 'scenic_spot_800-1000sqm',
                             'scenic_spot_1000-10000sqm', 'scenic_spot>1000sqm']
scenic_spot_area = scenic_spot_area.drop(columns=['SA2 name'])
sum_area(scenic_spot_area)

commerical_area = pd.read_excel("../data/raw/functional.xlsx",sheet_name="Table 8",header=5)
commerical_area.columns = ['SA2_Code', 'SA2 name','commerical_area<200sqm', 'commerical_area_200-400sqm',
                           'commerical_area_400-600sqm', 'commerical_area_600-800sqm', 'commerical_area_800-1000sqm',
                           'commerical_area_1000-10000sqm', 'commerical_area>1000sqm']
commerical_area = commerical_area.drop(columns=['SA2 name'])
sum_area(commerical_area)

utilities_area = pd.read_excel("../data/raw/functional.xlsx",sheet_name="Table 10",header=5)
utilities_area.columns = ['SA2_Code', 'SA2 name','utilities_area<200sqm', 'utilities_area_200-400sqm',
                          'utilities_area_400-600sqm', 'utilities_area_600-800sqm', 'utilities_area_800-1000sqm',
                          'utilities_area_1000-10000sqm', 'utilities_area>1000sqm']
utilities_area = utilities_area.drop(columns=['SA2 name'])
sum_area(utilities_area)

,SA2_Code,utilities_area<200sqm,utilities_area_200-400sqm,utilities_area_400-600sqm,utilities_area_600-800sqm,utilities_area_800-1000sqm,utilities_area_1000-10000sqm,utilities_area>1000sqm,total
0,4001,21,23,14,13,11,45,114,241
1,403041071,0,0,0,0,0,0,0,0
2,401011001,0,0,0,0,0,0,0,0
3,404031104,0,0,0,0,0,0,0,0
4,401021004,0,0,0,0,0,3,2,5
...,...,...,...,...,...,...,...,...,...
1518,107031143,0,0,4,1,0,4,1,10
1519,107011547,0,0,1,0,0,0,2,3
1520,107041548,4,0,0,0,4,3,4,15
1521,107041549,1,2,0,1,0,3,4,11


In [12]:
#sum the functional area by SA2
functional = pd.DataFrame()
functional["SA2_Code"] = residential_area["SA2_Code"]
functional["residential"] = residential_area["total"]
functional["primary_production"] = primary_production_area["total"]
functional["community_infrastructure"] = community_infrastructure_area["total"]
functional["mix_use"] = mix_use_area["total"]
functional["industrial"] = industrial_area["total"]
functional["scenic_spot"] = scenic_spot_area["total"]
functional["commerical"] = commerical_area["total"]
functional["utilities"] = utilities_area["total"]

In [13]:
# merge the exrternal with functional area data
external = pd.merge(external, functional, on='SA2_Code', how='left').fillna(np.nan)
external

,postcode,locality,state,LGA,SA3_NAME_2016,SA2_Code,SA2_Name,2001_population,2002_population,2003_population,...,2017-18_mean_income,2018-19_mean_income,residential,primary_production,community_infrastructure,mix_use,industrial,scenic_spot,commerical,utilities
0,3000,MELBOURNE,VIC,Melbourne,Melbourne City,206041122.0,Melbourne,NaN,NaN,NaN,...,51528.0,52476.0,0.0,0.0,0.0,1425.0,0.0,0.0,0.0,0.0
1,3001,MELBOURNE,VIC,Moonee Valley,Melbourne City,206041122.0,Melbourne,NaN,NaN,NaN,...,51528.0,52476.0,0.0,0.0,0.0,1425.0,0.0,0.0,0.0,0.0
2,3002,EAST MELBOURNE,VIC,Yarra,Melbourne City,206041119.0,East Melbourne,3731.0,3859.0,4243.0,...,128069.0,132130.0,632.0,0.0,0.0,35.0,0.0,0.0,153.0,33.0
3,3003,WEST MELBOURNE,VIC,Melbourne,Melbourne City,206041127.0,West Melbourne,0.0,0.0,0.0,...,237158.0,112680.0,0.0,0.0,0.0,8.0,81.0,0.0,19.0,74.0
4,3004,MELBOURNE,VIC,Yarra,Melbourne City,206041126.0,Southbank,NaN,NaN,NaN,...,73634.0,75071.0,2.0,0.0,0.0,186.0,0.0,0.0,3.0,40.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3529,8111,MELBOURNE,VIC,Yarra,Melbourne City,206041122.0,Melbourne,NaN,NaN,NaN,...,51528.0,52476.0,0.0,0.0,0.0,1425.0,0.0,0.0,0.0,0.0
3530,8120,MELBOURNE,VIC,Yarra,Melbourne City,206041122.0,Melbourne,NaN,NaN,NaN,...,51528.0,52476.0,0.0,0.0,0.0,1425.0,0.0,0.0,0.0,0.0
3531,8205,MELBOURNE,VIC,Yarra,Melbourne City,206041122.0,Melbourne,NaN,NaN,NaN,...,51528.0,52476.0,0.0,0.0,0.0,1425.0,0.0,0.0,0.0,0.0
3532,8785,DANDENONG,VIC,Monash,Dandenong,212041311.0,Dandenong,NaN,NaN,NaN,...,44205.0,45908.0,4846.0,76.0,76.0,27.0,2286.0,24.0,428.0,221.0


In [14]:
# calculate the number of cemetery by postcode
cemetery = pd.read_csv("../data/raw/cemetery.csv")
cemetery_count=pd.DataFrame.from_dict(dict(cemetery["Postcode"].value_counts()),orient='index').reset_index()
cemetery_count.columns=["postcode","cemetery_number"]

In [15]:
# merge the exrternal with cemetery data
external = pd.merge(external, cemetery_count, on='postcode', how='left').fillna(np.nan)
external

,postcode,locality,state,LGA,SA3_NAME_2016,SA2_Code,SA2_Name,2001_population,2002_population,2003_population,...,2018-19_mean_income,residential,primary_production,community_infrastructure,mix_use,industrial,scenic_spot,commerical,utilities,cemetery_number
0,3000,MELBOURNE,VIC,Melbourne,Melbourne City,206041122.0,Melbourne,NaN,NaN,NaN,...,52476.0,0.0,0.0,0.0,1425.0,0.0,0.0,0.0,0.0,NaN
1,3001,MELBOURNE,VIC,Moonee Valley,Melbourne City,206041122.0,Melbourne,NaN,NaN,NaN,...,52476.0,0.0,0.0,0.0,1425.0,0.0,0.0,0.0,0.0,NaN
2,3002,EAST MELBOURNE,VIC,Yarra,Melbourne City,206041119.0,East Melbourne,3731.0,3859.0,4243.0,...,132130.0,632.0,0.0,0.0,35.0,0.0,0.0,153.0,33.0,NaN
3,3003,WEST MELBOURNE,VIC,Melbourne,Melbourne City,206041127.0,West Melbourne,0.0,0.0,0.0,...,112680.0,0.0,0.0,0.0,8.0,81.0,0.0,19.0,74.0,NaN
4,3004,MELBOURNE,VIC,Yarra,Melbourne City,206041126.0,Southbank,NaN,NaN,NaN,...,75071.0,2.0,0.0,0.0,186.0,0.0,0.0,3.0,40.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3529,8111,MELBOURNE,VIC,Yarra,Melbourne City,206041122.0,Melbourne,NaN,NaN,NaN,...,52476.0,0.0,0.0,0.0,1425.0,0.0,0.0,0.0,0.0,NaN
3530,8120,MELBOURNE,VIC,Yarra,Melbourne City,206041122.0,Melbourne,NaN,NaN,NaN,...,52476.0,0.0,0.0,0.0,1425.0,0.0,0.0,0.0,0.0,NaN
3531,8205,MELBOURNE,VIC,Yarra,Melbourne City,206041122.0,Melbourne,NaN,NaN,NaN,...,52476.0,0.0,0.0,0.0,1425.0,0.0,0.0,0.0,0.0,NaN
3532,8785,DANDENONG,VIC,Monash,Dandenong,212041311.0,Dandenong,NaN,NaN,NaN,...,45908.0,4846.0,76.0,76.0,27.0,2286.0,24.0,428.0,221.0,NaN


In [16]:
# calculate the number of Sport&Recreational Facilities by postcode
facilities= pd.read_excel("../data/raw/Sport_Recreational_Facilities_list.xlsx")
facilities = facilities[['Facility Name','Pcode']]
facilities=facilities.drop_duplicates()
facilities_count=pd.DataFrame.from_dict(dict(facilities['Pcode'].value_counts()),orient='index').reset_index()
facilities_count.columns=["postcode","Sport&Recreational_facilities_number"]

In [17]:
# merge the exrternal with Sport&Recreational Facilitiesdata
external = pd.merge(external, facilities_count, on='postcode', how='left').fillna(np.nan)
external

,postcode,locality,state,LGA,SA3_NAME_2016,SA2_Code,SA2_Name,2001_population,2002_population,2003_population,...,residential,primary_production,community_infrastructure,mix_use,industrial,scenic_spot,commerical,utilities,cemetery_number,Sport&Recreational_facilities_number
0,3000,MELBOURNE,VIC,Melbourne,Melbourne City,206041122.0,Melbourne,NaN,NaN,NaN,...,0.0,0.0,0.0,1425.0,0.0,0.0,0.0,0.0,NaN,5.0
1,3001,MELBOURNE,VIC,Moonee Valley,Melbourne City,206041122.0,Melbourne,NaN,NaN,NaN,...,0.0,0.0,0.0,1425.0,0.0,0.0,0.0,0.0,NaN,1.0
2,3002,EAST MELBOURNE,VIC,Yarra,Melbourne City,206041119.0,East Melbourne,3731.0,3859.0,4243.0,...,632.0,0.0,0.0,35.0,0.0,0.0,153.0,33.0,NaN,NaN
3,3003,WEST MELBOURNE,VIC,Melbourne,Melbourne City,206041127.0,West Melbourne,0.0,0.0,0.0,...,0.0,0.0,0.0,8.0,81.0,0.0,19.0,74.0,NaN,NaN
4,3004,MELBOURNE,VIC,Yarra,Melbourne City,206041126.0,Southbank,NaN,NaN,NaN,...,2.0,0.0,0.0,186.0,0.0,0.0,3.0,40.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3529,8111,MELBOURNE,VIC,Yarra,Melbourne City,206041122.0,Melbourne,NaN,NaN,NaN,...,0.0,0.0,0.0,1425.0,0.0,0.0,0.0,0.0,NaN,NaN
3530,8120,MELBOURNE,VIC,Yarra,Melbourne City,206041122.0,Melbourne,NaN,NaN,NaN,...,0.0,0.0,0.0,1425.0,0.0,0.0,0.0,0.0,NaN,NaN
3531,8205,MELBOURNE,VIC,Yarra,Melbourne City,206041122.0,Melbourne,NaN,NaN,NaN,...,0.0,0.0,0.0,1425.0,0.0,0.0,0.0,0.0,NaN,NaN
3532,8785,DANDENONG,VIC,Monash,Dandenong,212041311.0,Dandenong,NaN,NaN,NaN,...,4846.0,76.0,76.0,27.0,2286.0,24.0,428.0,221.0,NaN,NaN


In [26]:
# read median house price time series data
house_price_yearly= pd.read_excel("../data/raw/Median_Suburb_Time_2021.xls",sheet_name="Sheet1",header=1)
house_price_yearly.columns =["locality","2010 Median","2011 Median","2012 Median","2013 Median","2014 Median",
                             "2015 Median","2016 Median","2017 Median","2018 Median","2019 Median","2020 Median",
                             "prelim 2021 Median","change 2019-2020","change 2010-2020","Growth PA"]
house_price_yearly = house_price_yearly.iloc[1: , :]
house_price_yearly

,locality,2010 Median,2011 Median,2012 Median,2013 Median,2014 Median,2015 Median,2016 Median,2017 Median,2018 Median,2019 Median,2020 Median,prelim 2021 Median,change 2019-2020,change 2010-2020,Growth PA
1,ABBOTSFORD,736000,730000,714000,792500,862500,925000,1187500,1280000,1192500,1050000,1205000,1375000,15,64,5.1
2,ABERFELDIE,1046500,994000,852500,947500,1045000,1207500,1300000,1471000,1500000,1390000,1620000,1700000,17,55,4.5
3,AINTREE,-,-,-,-,-,590000,607500,571000,568500,593000,685000,717500,16,NaN,NaN
4,AIREYS INLET,606000,680000,634000,664000,625500,680000,715000,737500,869000,985000,1125000,1522000,14,86,6.4
5,AIRPORT WEST,575000,557500,495000,532000,575000,635000,742000,845000,845000,795000,807500,889500,2,40,3.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
782,YARRAM,166000,195500,215000,190000,202500,245000,230000,225000,218000,240000,260000,343000,8,57,4.6
783,YARRAVILLE,647000,607000,613000,696000,710000,810000,900500,1000000,1045000,974000,1038500,1017500,7,61,4.8
784,YARRAWONGA,297000,270000,300000,302500,297000,300500,320000,324500,345000,375000,451000,539500,20,52,4.3
785,YEA,240000,270000,255000,260000,255000,260000,305000,386500,392500,441000,422500,515000,-4,76,5.8


In [37]:
external_locality = list(external["locality"].unique())
house_price_locality = list(house_price_yearly["locality"].unique())

common_locality = house_price_yearly["locality"].isin(external["SA2_Name"])
common_locality

1      False
2      False
3      False
4      False
5      False
       ...  
782    False
783    False
784    False
785    False
786    False
Name: locality, Length: 786, dtype: bool

In [63]:
# read affordable letting data
affordable_1br= pd.read_excel("../data/raw/Affordable_Lettings_2021.xlsx",sheet_name="lga aff 1br",header=2)
affordable_1br.columns = ['LGA', 'Affordable_1r_Mar_2000', 'Percent_1r_Mar_2000', 'Affordable_1r_Jun_2000', 'Percent_1r_Jun_2000', 'Affordable_1r_Sep_2000', 'Percent_1r_Sep_2000', 'Affordable_1r_Dec_2000', 'Percent_1r_Dec_2000', 'Affordable_1r_Mar_2001', 'Percent_1r_Mar_2001', 'Affordable_1r_Jun_2001', 'Percent_1r_Jun_2001', 'Affordable_1r_Sep_2001', 'Percent_1r_Sep_2001', 'Affordable_1r_Dec_2001', 'Percent_1r_Dec_2001', 'Affordable_1r_Mar_2002', 'Percent_1r_Mar_2002', 'Affordable_1r_Jun_2002', 'Percent_1r_Jun_2002', 'Affordable_1r_Sep_2002', 'Percent_1r_Sep_2002', 'Affordable_1r_Dec_2002', 'Percent_1r_Dec_2002', 'Affordable_1r_Mar_2003', 'Percent_1r_Mar_2003', 'Affordable_1r_Jun_2003', 'Percent_1r_Jun_2003', 'Affordable_1r_Sep_2003', 'Percent_1r_Sep_2003', 'Affordable_1r_Dec_2003', 'Percent_1r_Dec_2003', 'Affordable_1r_Mar_2004', 'Percent_1r_Mar_2004', 'Affordable_1r_Jun_2004', 'Percent_1r_Jun_2004', 'Affordable_1r_Sep_2004', 'Percent_1r_Sep_2004', 'Affordable_1r_Dec_2004', 'Percent_1r_Dec_2004', 'Affordable_1r_Mar_2005', 'Percent_1r_Mar_2005', 'Affordable_1r_Jun_2005', 'Percent_1r_Jun_2005', 'Affordable_1r_Sep_2005', 'Percent_1r_Sep_2005', 'Affordable_1r_Dec_2005', 'Percent_1r_Dec_2005', 'Affordable_1r_Mar_2006', 'Percent_1r_Mar_2006', 'Affordable_1r_Jun_2006', 'Percent_1r_Jun_2006', 'Affordable_1r_Sep_2006', 'Percent_1r_Sep_2006', 'Affordable_1r_Dec_2006', 'Percent_1r_Dec_2006', 'Affordable_1r_Mar_2007', 'Percent_1r_Mar_2007', 'Affordable_1r_Jun_2007', 'Percent_1r_Jun_2007', 'Affordable_1r_Sep_2007', 'Percent_1r_Sep_2007', 'Affordable_1r_Dec_2007', 'Percent_1r_Dec_2007', 'Affordable_1r_Mar_2008', 'Percent_1r_Mar_2008', 'Affordable_1r_Jun_2008', 'Percent_1r_Jun_2008', 'Affordable_1r_Sep_2008', 'Percent_1r_Sep_2008', 'Affordable_1r_Dec_2008', 'Percent_1r_Dec_2008', 'Affordable_1r_Mar_2009', 'Percent_1r_Mar_2009', 'Affordable_1r_Jun_2009', 'Percent_1r_Jun_2009', 'Affordable_1r_Sep_2009', 'Percent_1r_Sep_2009', 'Affordable_1r_Dec_2009', 'Percent_1r_Dec_2009', 'Affordable_1r_Mar_2010', 'Percent_1r_Mar_2010', 'Affordable_1r_Jun_2010', 'Percent_1r_Jun_2010', 'Affordable_1r_Sep_2010', 'Percent_1r_Sep_2010', 'Affordable_1r_Dec_2010', 'Percent_1r_Dec_2010', 'Affordable_1r_Mar_2011', 'Percent_1r_Mar_2011', 'Affordable_1r_Jun_2011', 'Percent_1r_Jun_2011', 'Affordable_1r_Sep_2011', 'Percent_1r_Sep_2011', 'Affordable_1r_Dec_2011', 'Percent_1r_Dec_2011', 'Affordable_1r_Mar_2012', 'Percent_1r_Mar_2012', 'Affordable_1r_Jun_2012', 'Percent_1r_Jun_2012', 'Affordable_1r_Sep_2012', 'Percent_1r_Sep_2012', 'Affordable_1r_Dec_2012', 'Percent_1r_Dec_2012', 'Affordable_1r_Mar_2013', 'Percent_1r_Mar_2013', 'Affordable_1r_Jun_2013', 'Percent_1r_Jun_2013', 'Affordable_1r_Sep_2013', 'Percent_1r_Sep_2013', 'Affordable_1r_Dec_2013', 'Percent_1r_Dec_2013', 'Affordable_1r_Mar_2014', 'Percent_1r_Mar_2014', 'Affordable_1r_Jun_2014', 'Percent_1r_Jun_2014', 'Affordable_1r_Sep_2014', 'Percent_1r_Sep_2014', 'Affordable_1r_Dec_2014', 'Percent_1r_Dec_2014', 'Affordable_1r_Mar_2015', 'Percent_1r_Mar_2015', 'Affordable_1r_Jun_2015', 'Percent_1r_Jun_2015', 'Affordable_1r_Sep_2015', 'Percent_1r_Sep_2015', 'Affordable_1r_Dec_2015', 'Percent_1r_Dec_2015', 'Affordable_1r_Mar_2016', 'Percent_1r_Mar_2016', 'Affordable_1r_Jun_2016', 'Percent_1r_Jun_2016', 'Affordable_1r_Sep_2016', 'Percent_1r_Sep_2016', 'Affordable_1r_Dec_2016', 'Percent_1r_Dec_2016', 'Affordable_1r_Mar_2017', 'Percent_1r_Mar_2017', 'Affordable_1r_Jun_2017', 'Percent_1r_Jun_2017', 'Affordable_1r_Sep_2017', 'Percent_1r_Sep_2017', 'Affordable_1r_Dec_2017', 'Percent_1r_Dec_2017', 'Affordable_1r_Mar_2018', 'Percent_1r_Mar_2018', 'Affordable_1r_Jun_2018', 'Percent_1r_Jun_2018', 'Affordable_1r_Sep_2018', 'Percent_1r_Sep_2018', 'Affordable_1r_Dec_2018', 'Percent_1r_Dec_2018', 'Affordable_1r_Mar_2019', 'Percent_1r_Mar_2019', 'Affordable_1r_Jun_2019', 'Percent_1r_Jun_2019', 'Affordable_1r_Sep_2019', 'Percent_1r_Sep_2019', 'Affordable_1r_Dec_2019', 'Percent_1r_Dec_2019', 'Affordable_1r_Mar_2020', 'Percent_1r_Mar_2020', 'Affordable_1r_Jun_2020', 'Percent_1r_Jun_2020', 'Affordable_1r_Sep_2020', 'Percent_1r_Sep_2020', 'Affordable_1r_Dec_2020', 'Percent_1r_Dec_2020', 'Affordable_1r_Mar_2021', 'Percent_1r_Mar_2021', 'Affordable_1r_Jun_2021', 'Percent_1r_Jun_2021', 'Affordable_1r_Sep_2021', 'Percent_1r_Sep_2021']

affordable_2br= pd.read_excel("../data/raw/Affordable_Lettings_2021.xlsx",sheet_name="lga aff 2br",header=2)
affordable_2br.columns = ['LGA', 'Affordable_2r_Mar_2000', 'Percent_2r_Mar_2000', 'Affordable_2r_Jun_2000', 'Percent_2r_Jun_2000', 'Affordable_2r_Sep_2000', 'Percent_2r_Sep_2000', 'Affordable_2r_Dec_2000', 'Percent_2r_Dec_2000', 'Affordable_2r_Mar_2001', 'Percent_2r_Mar_2001', 'Affordable_2r_Jun_2001', 'Percent_2r_Jun_2001', 'Affordable_2r_Sep_2001', 'Percent_2r_Sep_2001', 'Affordable_2r_Dec_2001', 'Percent_2r_Dec_2001', 'Affordable_2r_Mar_2002', 'Percent_2r_Mar_2002', 'Affordable_2r_Jun_2002', 'Percent_2r_Jun_2002', 'Affordable_2r_Sep_2002', 'Percent_2r_Sep_2002', 'Affordable_2r_Dec_2002', 'Percent_2r_Dec_2002', 'Affordable_2r_Mar_2003', 'Percent_2r_Mar_2003', 'Affordable_2r_Jun_2003', 'Percent_2r_Jun_2003', 'Affordable_2r_Sep_2003', 'Percent_2r_Sep_2003', 'Affordable_2r_Dec_2003', 'Percent_2r_Dec_2003', 'Affordable_2r_Mar_2004', 'Percent_2r_Mar_2004', 'Affordable_2r_Jun_2004', 'Percent_2r_Jun_2004', 'Affordable_2r_Sep_2004', 'Percent_2r_Sep_2004', 'Affordable_2r_Dec_2004', 'Percent_2r_Dec_2004', 'Affordable_2r_Mar_2005', 'Percent_2r_Mar_2005', 'Affordable_2r_Jun_2005', 'Percent_2r_Jun_2005', 'Affordable_2r_Sep_2005', 'Percent_2r_Sep_2005', 'Affordable_2r_Dec_2005', 'Percent_2r_Dec_2005', 'Affordable_2r_Mar_2006', 'Percent_2r_Mar_2006', 'Affordable_2r_Jun_2006', 'Percent_2r_Jun_2006', 'Affordable_2r_Sep_2006', 'Percent_2r_Sep_2006', 'Affordable_2r_Dec_2006', 'Percent_2r_Dec_2006', 'Affordable_2r_Mar_2007', 'Percent_2r_Mar_2007', 'Affordable_2r_Jun_2007', 'Percent_2r_Jun_2007', 'Affordable_2r_Sep_2007', 'Percent_2r_Sep_2007', 'Affordable_2r_Dec_2007', 'Percent_2r_Dec_2007', 'Affordable_2r_Mar_2008', 'Percent_2r_Mar_2008', 'Affordable_2r_Jun_2008', 'Percent_2r_Jun_2008', 'Affordable_2r_Sep_2008', 'Percent_2r_Sep_2008', 'Affordable_2r_Dec_2008', 'Percent_2r_Dec_2008', 'Affordable_2r_Mar_2009', 'Percent_2r_Mar_2009', 'Affordable_2r_Jun_2009', 'Percent_2r_Jun_2009', 'Affordable_2r_Sep_2009', 'Percent_2r_Sep_2009', 'Affordable_2r_Dec_2009', 'Percent_2r_Dec_2009', 'Affordable_2r_Mar_2010', 'Percent_2r_Mar_2010', 'Affordable_2r_Jun_2010', 'Percent_2r_Jun_2010', 'Affordable_2r_Sep_2010', 'Percent_2r_Sep_2010', 'Affordable_2r_Dec_2010', 'Percent_2r_Dec_2010', 'Affordable_2r_Mar_2011', 'Percent_2r_Mar_2011', 'Affordable_2r_Jun_2011', 'Percent_2r_Jun_2011', 'Affordable_2r_Sep_2011', 'Percent_2r_Sep_2011', 'Affordable_2r_Dec_2011', 'Percent_2r_Dec_2011', 'Affordable_2r_Mar_2012', 'Percent_2r_Mar_2012', 'Affordable_2r_Jun_2012', 'Percent_2r_Jun_2012', 'Affordable_2r_Sep_2012', 'Percent_2r_Sep_2012', 'Affordable_2r_Dec_2012', 'Percent_2r_Dec_2012', 'Affordable_2r_Mar_2013', 'Percent_2r_Mar_2013', 'Affordable_2r_Jun_2013', 'Percent_2r_Jun_2013', 'Affordable_2r_Sep_2013', 'Percent_2r_Sep_2013', 'Affordable_2r_Dec_2013', 'Percent_2r_Dec_2013', 'Affordable_2r_Mar_2014', 'Percent_2r_Mar_2014', 'Affordable_2r_Jun_2014', 'Percent_2r_Jun_2014', 'Affordable_2r_Sep_2014', 'Percent_2r_Sep_2014', 'Affordable_2r_Dec_2014', 'Percent_2r_Dec_2014', 'Affordable_2r_Mar_2015', 'Percent_2r_Mar_2015', 'Affordable_2r_Jun_2015', 'Percent_2r_Jun_2015', 'Affordable_2r_Sep_2015', 'Percent_2r_Sep_2015', 'Affordable_2r_Dec_2015', 'Percent_2r_Dec_2015', 'Affordable_2r_Mar_2016', 'Percent_2r_Mar_2016', 'Affordable_2r_Jun_2016', 'Percent_2r_Jun_2016', 'Affordable_2r_Sep_2016', 'Percent_2r_Sep_2016', 'Affordable_2r_Dec_2016', 'Percent_2r_Dec_2016', 'Affordable_2r_Mar_2017', 'Percent_2r_Mar_2017', 'Affordable_2r_Jun_2017', 'Percent_2r_Jun_2017', 'Affordable_2r_Sep_2017', 'Percent_2r_Sep_2017', 'Affordable_2r_Dec_2017', 'Percent_2r_Dec_2017', 'Affordable_2r_Mar_2018', 'Percent_2r_Mar_2018', 'Affordable_2r_Jun_2018', 'Percent_2r_Jun_2018', 'Affordable_2r_Sep_2018', 'Percent_2r_Sep_2018', 'Affordable_2r_Dec_2018', 'Percent_2r_Dec_2018', 'Affordable_2r_Mar_2019', 'Percent_2r_Mar_2019', 'Affordable_2r_Jun_2019', 'Percent_2r_Jun_2019', 'Affordable_2r_Sep_2019', 'Percent_2r_Sep_2019', 'Affordable_2r_Dec_2019', 'Percent_2r_Dec_2019', 'Affordable_2r_Mar_2020', 'Percent_2r_Mar_2020', 'Affordable_2r_Jun_2020', 'Percent_2r_Jun_2020', 'Affordable_2r_Sep_2020', 'Percent_2r_Sep_2020', 'Affordable_2r_Dec_2020', 'Percent_2r_Dec_2020', 'Affordable_2r_Mar_2021', 'Percent_2r_Mar_2021', 'Affordable_2r_Jun_2021', 'Percent_2r_Jun_2021', 'Affordable_2r_Sep_2021', 'Percent_2r_Sep_2021']

affordable_3br= pd.read_excel("../data/raw/Affordable_Lettings_2021.xlsx",sheet_name="lga aff 3br",header=2)
affordable_3br.columns = ['LGA', 'Affordable_3r_Mar_2000', 'Percent_3r_Mar_2000', 'Affordable_3r_Jun_2000', 'Percent_3r_Jun_2000', 'Affordable_3r_Sep_2000', 'Percent_3r_Sep_2000', 'Affordable_3r_Dec_2000', 'Percent_3r_Dec_2000', 'Affordable_3r_Mar_2001', 'Percent_3r_Mar_2001', 'Affordable_3r_Jun_2001', 'Percent_3r_Jun_2001', 'Affordable_3r_Sep_2001', 'Percent_3r_Sep_2001', 'Affordable_3r_Dec_2001', 'Percent_3r_Dec_2001', 'Affordable_3r_Mar_2002', 'Percent_3r_Mar_2002', 'Affordable_3r_Jun_2002', 'Percent_3r_Jun_2002', 'Affordable_3r_Sep_2002', 'Percent_3r_Sep_2002', 'Affordable_3r_Dec_2002', 'Percent_3r_Dec_2002', 'Affordable_3r_Mar_2003', 'Percent_3r_Mar_2003', 'Affordable_3r_Jun_2003', 'Percent_3r_Jun_2003', 'Affordable_3r_Sep_2003', 'Percent_3r_Sep_2003', 'Affordable_3r_Dec_2003', 'Percent_3r_Dec_2003', 'Affordable_3r_Mar_2004', 'Percent_3r_Mar_2004', 'Affordable_3r_Jun_2004', 'Percent_3r_Jun_2004', 'Affordable_3r_Sep_2004', 'Percent_3r_Sep_2004', 'Affordable_3r_Dec_2004', 'Percent_3r_Dec_2004', 'Affordable_3r_Mar_2005', 'Percent_3r_Mar_2005', 'Affordable_3r_Jun_2005', 'Percent_3r_Jun_2005', 'Affordable_3r_Sep_2005', 'Percent_3r_Sep_2005', 'Affordable_3r_Dec_2005', 'Percent_3r_Dec_2005', 'Affordable_3r_Mar_2006', 'Percent_3r_Mar_2006', 'Affordable_3r_Jun_2006', 'Percent_3r_Jun_2006', 'Affordable_3r_Sep_2006', 'Percent_3r_Sep_2006', 'Affordable_3r_Dec_2006', 'Percent_3r_Dec_2006', 'Affordable_3r_Mar_2007', 'Percent_3r_Mar_2007', 'Affordable_3r_Jun_2007', 'Percent_3r_Jun_2007', 'Affordable_3r_Sep_2007', 'Percent_3r_Sep_2007', 'Affordable_3r_Dec_2007', 'Percent_3r_Dec_2007', 'Affordable_3r_Mar_2008', 'Percent_3r_Mar_2008', 'Affordable_3r_Jun_2008', 'Percent_3r_Jun_2008', 'Affordable_3r_Sep_2008', 'Percent_3r_Sep_2008', 'Affordable_3r_Dec_2008', 'Percent_3r_Dec_2008', 'Affordable_3r_Mar_2009', 'Percent_3r_Mar_2009', 'Affordable_3r_Jun_2009', 'Percent_3r_Jun_2009', 'Affordable_3r_Sep_2009', 'Percent_3r_Sep_2009', 'Affordable_3r_Dec_2009', 'Percent_3r_Dec_2009', 'Affordable_3r_Mar_2010', 'Percent_3r_Mar_2010', 'Affordable_3r_Jun_2010', 'Percent_3r_Jun_2010', 'Affordable_3r_Sep_2010', 'Percent_3r_Sep_2010', 'Affordable_3r_Dec_2010', 'Percent_3r_Dec_2010', 'Affordable_3r_Mar_2011', 'Percent_3r_Mar_2011', 'Affordable_3r_Jun_2011', 'Percent_3r_Jun_2011', 'Affordable_3r_Sep_2011', 'Percent_3r_Sep_2011', 'Affordable_3r_Dec_2011', 'Percent_3r_Dec_2011', 'Affordable_3r_Mar_2012', 'Percent_3r_Mar_2012', 'Affordable_3r_Jun_2012', 'Percent_3r_Jun_2012', 'Affordable_3r_Sep_2012', 'Percent_3r_Sep_2012', 'Affordable_3r_Dec_2012', 'Percent_3r_Dec_2012', 'Affordable_3r_Mar_2013', 'Percent_3r_Mar_2013', 'Affordable_3r_Jun_2013', 'Percent_3r_Jun_2013', 'Affordable_3r_Sep_2013', 'Percent_3r_Sep_2013', 'Affordable_3r_Dec_2013', 'Percent_3r_Dec_2013', 'Affordable_3r_Mar_2014', 'Percent_3r_Mar_2014', 'Affordable_3r_Jun_2014', 'Percent_3r_Jun_2014', 'Affordable_3r_Sep_2014', 'Percent_3r_Sep_2014', 'Affordable_3r_Dec_2014', 'Percent_3r_Dec_2014', 'Affordable_3r_Mar_2015', 'Percent_3r_Mar_2015', 'Affordable_3r_Jun_2015', 'Percent_3r_Jun_2015', 'Affordable_3r_Sep_2015', 'Percent_3r_Sep_2015', 'Affordable_3r_Dec_2015', 'Percent_3r_Dec_2015', 'Affordable_3r_Mar_2016', 'Percent_3r_Mar_2016', 'Affordable_3r_Jun_2016', 'Percent_3r_Jun_2016', 'Affordable_3r_Sep_2016', 'Percent_3r_Sep_2016', 'Affordable_3r_Dec_2016', 'Percent_3r_Dec_2016', 'Affordable_3r_Mar_2017', 'Percent_3r_Mar_2017', 'Affordable_3r_Jun_2017', 'Percent_3r_Jun_2017', 'Affordable_3r_Sep_2017', 'Percent_3r_Sep_2017', 'Affordable_3r_Dec_2017', 'Percent_3r_Dec_2017', 'Affordable_3r_Mar_2018', 'Percent_3r_Mar_2018', 'Affordable_3r_Jun_2018', 'Percent_3r_Jun_2018', 'Affordable_3r_Sep_2018', 'Percent_3r_Sep_2018', 'Affordable_3r_Dec_2018', 'Percent_3r_Dec_2018', 'Affordable_3r_Mar_2019', 'Percent_3r_Mar_2019', 'Affordable_3r_Jun_2019', 'Percent_3r_Jun_2019', 'Affordable_3r_Sep_2019', 'Percent_3r_Sep_2019', 'Affordable_3r_Dec_2019', 'Percent_3r_Dec_2019', 'Affordable_3r_Mar_2020', 'Percent_3r_Mar_2020', 'Affordable_3r_Jun_2020', 'Percent_3r_Jun_2020', 'Affordable_3r_Sep_2020', 'Percent_3r_Sep_2020', 'Affordable_3r_Dec_2020', 'Percent_3r_Dec_2020', 'Affordable_3r_Mar_2021', 'Percent_3r_Mar_2021', 'Affordable_3r_Jun_2021', 'Percent_3r_Jun_2021', 'Affordable_3r_Sep_2021', 'Percent_3r_Sep_2021']

affordable_4br= pd.read_excel("../data/raw/Affordable_Lettings_2021.xlsx",sheet_name="lga aff 4br",header=2)
affordable_4br.columns = ['LGA', 'Affordable_4r_Mar_2000', 'Percent_4r_Mar_2000', 'Affordable_4r_Jun_2000', 'Percent_4r_Jun_2000', 'Affordable_4r_Sep_2000', 'Percent_4r_Sep_2000', 'Affordable_4r_Dec_2000', 'Percent_4r_Dec_2000', 'Affordable_4r_Mar_2001', 'Percent_4r_Mar_2001', 'Affordable_4r_Jun_2001', 'Percent_4r_Jun_2001', 'Affordable_4r_Sep_2001', 'Percent_4r_Sep_2001', 'Affordable_4r_Dec_2001', 'Percent_4r_Dec_2001', 'Affordable_4r_Mar_2002', 'Percent_4r_Mar_2002', 'Affordable_4r_Jun_2002', 'Percent_4r_Jun_2002', 'Affordable_4r_Sep_2002', 'Percent_4r_Sep_2002', 'Affordable_4r_Dec_2002', 'Percent_4r_Dec_2002', 'Affordable_4r_Mar_2003', 'Percent_4r_Mar_2003', 'Affordable_4r_Jun_2003', 'Percent_4r_Jun_2003', 'Affordable_4r_Sep_2003', 'Percent_4r_Sep_2003', 'Affordable_4r_Dec_2003', 'Percent_4r_Dec_2003', 'Affordable_4r_Mar_2004', 'Percent_4r_Mar_2004', 'Affordable_4r_Jun_2004', 'Percent_4r_Jun_2004', 'Affordable_4r_Sep_2004', 'Percent_4r_Sep_2004', 'Affordable_4r_Dec_2004', 'Percent_4r_Dec_2004', 'Affordable_4r_Mar_2005', 'Percent_4r_Mar_2005', 'Affordable_4r_Jun_2005', 'Percent_4r_Jun_2005', 'Affordable_4r_Sep_2005', 'Percent_4r_Sep_2005', 'Affordable_4r_Dec_2005', 'Percent_4r_Dec_2005', 'Affordable_4r_Mar_2006', 'Percent_4r_Mar_2006', 'Affordable_4r_Jun_2006', 'Percent_4r_Jun_2006', 'Affordable_4r_Sep_2006', 'Percent_4r_Sep_2006', 'Affordable_4r_Dec_2006', 'Percent_4r_Dec_2006', 'Affordable_4r_Mar_2007', 'Percent_4r_Mar_2007', 'Affordable_4r_Jun_2007', 'Percent_4r_Jun_2007', 'Affordable_4r_Sep_2007', 'Percent_4r_Sep_2007', 'Affordable_4r_Dec_2007', 'Percent_4r_Dec_2007', 'Affordable_4r_Mar_2008', 'Percent_4r_Mar_2008', 'Affordable_4r_Jun_2008', 'Percent_4r_Jun_2008', 'Affordable_4r_Sep_2008', 'Percent_4r_Sep_2008', 'Affordable_4r_Dec_2008', 'Percent_4r_Dec_2008', 'Affordable_4r_Mar_2009', 'Percent_4r_Mar_2009', 'Affordable_4r_Jun_2009', 'Percent_4r_Jun_2009', 'Affordable_4r_Sep_2009', 'Percent_4r_Sep_2009', 'Affordable_4r_Dec_2009', 'Percent_4r_Dec_2009', 'Affordable_4r_Mar_2010', 'Percent_4r_Mar_2010', 'Affordable_4r_Jun_2010', 'Percent_4r_Jun_2010', 'Affordable_4r_Sep_2010', 'Percent_4r_Sep_2010', 'Affordable_4r_Dec_2010', 'Percent_4r_Dec_2010', 'Affordable_4r_Mar_2011', 'Percent_4r_Mar_2011', 'Affordable_4r_Jun_2011', 'Percent_4r_Jun_2011', 'Affordable_4r_Sep_2011', 'Percent_4r_Sep_2011', 'Affordable_4r_Dec_2011', 'Percent_4r_Dec_2011', 'Affordable_4r_Mar_2012', 'Percent_4r_Mar_2012', 'Affordable_4r_Jun_2012', 'Percent_4r_Jun_2012', 'Affordable_4r_Sep_2012', 'Percent_4r_Sep_2012', 'Affordable_4r_Dec_2012', 'Percent_4r_Dec_2012', 'Affordable_4r_Mar_2013', 'Percent_4r_Mar_2013', 'Affordable_4r_Jun_2013', 'Percent_4r_Jun_2013', 'Affordable_4r_Sep_2013', 'Percent_4r_Sep_2013', 'Affordable_4r_Dec_2013', 'Percent_4r_Dec_2013', 'Affordable_4r_Mar_2014', 'Percent_4r_Mar_2014', 'Affordable_4r_Jun_2014', 'Percent_4r_Jun_2014', 'Affordable_4r_Sep_2014', 'Percent_4r_Sep_2014', 'Affordable_4r_Dec_2014', 'Percent_4r_Dec_2014', 'Affordable_4r_Mar_2015', 'Percent_4r_Mar_2015', 'Affordable_4r_Jun_2015', 'Percent_4r_Jun_2015', 'Affordable_4r_Sep_2015', 'Percent_4r_Sep_2015', 'Affordable_4r_Dec_2015', 'Percent_4r_Dec_2015', 'Affordable_4r_Mar_2016', 'Percent_4r_Mar_2016', 'Affordable_4r_Jun_2016', 'Percent_4r_Jun_2016', 'Affordable_4r_Sep_2016', 'Percent_4r_Sep_2016', 'Affordable_4r_Dec_2016', 'Percent_4r_Dec_2016', 'Affordable_4r_Mar_2017', 'Percent_4r_Mar_2017', 'Affordable_4r_Jun_2017', 'Percent_4r_Jun_2017', 'Affordable_4r_Sep_2017', 'Percent_4r_Sep_2017', 'Affordable_4r_Dec_2017', 'Percent_4r_Dec_2017', 'Affordable_4r_Mar_2018', 'Percent_4r_Mar_2018', 'Affordable_4r_Jun_2018', 'Percent_4r_Jun_2018', 'Affordable_4r_Sep_2018', 'Percent_4r_Sep_2018', 'Affordable_4r_Dec_2018', 'Percent_4r_Dec_2018', 'Affordable_4r_Mar_2019', 'Percent_4r_Mar_2019', 'Affordable_4r_Jun_2019', 'Percent_4r_Jun_2019', 'Affordable_4r_Sep_2019', 'Percent_4r_Sep_2019', 'Affordable_4r_Dec_2019', 'Percent_4r_Dec_2019', 'Affordable_4r_Mar_2020', 'Percent_4r_Mar_2020', 'Affordable_4r_Jun_2020', 'Percent_4r_Jun_2020', 'Affordable_4r_Sep_2020', 'Percent_4r_Sep_2020', 'Affordable_4r_Dec_2020', 'Percent_4r_Dec_2020', 'Affordable_4r_Mar_2021', 'Percent_4r_Mar_2021', 'Affordable_4r_Jun_2021', 'Percent_4r_Jun_2021', 'Affordable_4r_Sep_2021', 'Percent_4r_Sep_2021']

affordable_total= pd.read_excel("../data/raw/Affordable_Lettings_2021.xlsx",sheet_name="lga aff total",header=2)
affordable_total.columns = ['LGA', 'Affordable_total_Mar_2000', 'Percent_total_Mar_2000', 'Affordable_total_Jun_2000', 'Percent_total_Jun_2000', 'Affordable_total_Sep_2000', 'Percent_total_Sep_2000', 'Affordable_total_Dec_2000', 'Percent_total_Dec_2000', 'Affordable_total_Mar_2001', 'Percent_total_Mar_2001', 'Affordable_total_Jun_2001', 'Percent_total_Jun_2001', 'Affordable_total_Sep_2001', 'Percent_total_Sep_2001', 'Affordable_total_Dec_2001', 'Percent_total_Dec_2001', 'Affordable_total_Mar_2002', 'Percent_total_Mar_2002', 'Affordable_total_Jun_2002', 'Percent_total_Jun_2002', 'Affordable_total_Sep_2002', 'Percent_total_Sep_2002', 'Affordable_total_Dec_2002', 'Percent_total_Dec_2002', 'Affordable_total_Mar_2003', 'Percent_total_Mar_2003', 'Affordable_total_Jun_2003', 'Percent_total_Jun_2003', 'Affordable_total_Sep_2003', 'Percent_total_Sep_2003', 'Affordable_total_Dec_2003', 'Percent_total_Dec_2003', 'Affordable_total_Mar_2004', 'Percent_total_Mar_2004', 'Affordable_total_Jun_2004', 'Percent_total_Jun_2004', 'Affordable_total_Sep_2004', 'Percent_total_Sep_2004', 'Affordable_total_Dec_2004', 'Percent_total_Dec_2004', 'Affordable_total_Mar_2005', 'Percent_total_Mar_2005', 'Affordable_total_Jun_2005', 'Percent_total_Jun_2005', 'Affordable_total_Sep_2005', 'Percent_total_Sep_2005', 'Affordable_total_Dec_2005', 'Percent_total_Dec_2005', 'Affordable_total_Mar_2006', 'Percent_total_Mar_2006', 'Affordable_total_Jun_2006', 'Percent_total_Jun_2006', 'Affordable_total_Sep_2006', 'Percent_total_Sep_2006', 'Affordable_total_Dec_2006', 'Percent_total_Dec_2006', 'Affordable_total_Mar_2007', 'Percent_total_Mar_2007', 'Affordable_total_Jun_2007', 'Percent_total_Jun_2007', 'Affordable_total_Sep_2007', 'Percent_total_Sep_2007', 'Affordable_total_Dec_2007', 'Percent_total_Dec_2007', 'Affordable_total_Mar_2008', 'Percent_total_Mar_2008', 'Affordable_total_Jun_2008', 'Percent_total_Jun_2008', 'Affordable_total_Sep_2008', 'Percent_total_Sep_2008', 'Affordable_total_Dec_2008', 'Percent_total_Dec_2008', 'Affordable_total_Mar_2009', 'Percent_total_Mar_2009', 'Affordable_total_Jun_2009', 'Percent_total_Jun_2009', 'Affordable_total_Sep_2009', 'Percent_total_Sep_2009', 'Affordable_total_Dec_2009', 'Percent_total_Dec_2009', 'Affordable_total_Mar_2010', 'Percent_total_Mar_2010', 'Affordable_total_Jun_2010', 'Percent_total_Jun_2010', 'Affordable_total_Sep_2010', 'Percent_total_Sep_2010', 'Affordable_total_Dec_2010', 'Percent_total_Dec_2010', 'Affordable_total_Mar_2011', 'Percent_total_Mar_2011', 'Affordable_total_Jun_2011', 'Percent_total_Jun_2011', 'Affordable_total_Sep_2011', 'Percent_total_Sep_2011', 'Affordable_total_Dec_2011', 'Percent_total_Dec_2011', 'Affordable_total_Mar_2012', 'Percent_total_Mar_2012', 'Affordable_total_Jun_2012', 'Percent_total_Jun_2012', 'Affordable_total_Sep_2012', 'Percent_total_Sep_2012', 'Affordable_total_Dec_2012', 'Percent_total_Dec_2012', 'Affordable_total_Mar_2013', 'Percent_total_Mar_2013', 'Affordable_total_Jun_2013', 'Percent_total_Jun_2013', 'Affordable_total_Sep_2013', 'Percent_total_Sep_2013', 'Affordable_total_Dec_2013', 'Percent_total_Dec_2013', 'Affordable_total_Mar_2014', 'Percent_total_Mar_2014', 'Affordable_total_Jun_2014', 'Percent_total_Jun_2014', 'Affordable_total_Sep_2014', 'Percent_total_Sep_2014', 'Affordable_total_Dec_2014', 'Percent_total_Dec_2014', 'Affordable_total_Mar_2015', 'Percent_total_Mar_2015', 'Affordable_total_Jun_2015', 'Percent_total_Jun_2015', 'Affordable_total_Sep_2015', 'Percent_total_Sep_2015', 'Affordable_total_Dec_2015', 'Percent_total_Dec_2015', 'Affordable_total_Mar_2016', 'Percent_total_Mar_2016', 'Affordable_total_Jun_2016', 'Percent_total_Jun_2016', 'Affordable_total_Sep_2016', 'Percent_total_Sep_2016', 'Affordable_total_Dec_2016', 'Percent_total_Dec_2016', 'Affordable_total_Mar_2017', 'Percent_total_Mar_2017', 'Affordable_total_Jun_2017', 'Percent_total_Jun_2017', 'Affordable_total_Sep_2017', 'Percent_total_Sep_2017', 'Affordable_total_Dec_2017', 'Percent_total_Dec_2017', 'Affordable_total_Mar_2018', 'Percent_total_Mar_2018', 'Affordable_total_Jun_2018', 'Percent_total_Jun_2018', 'Affordable_total_Sep_2018', 'Percent_total_Sep_2018', 'Affordable_total_Dec_2018', 'Percent_total_Dec_2018', 'Affordable_total_Mar_2019', 'Percent_total_Mar_2019', 'Affordable_total_Jun_2019', 'Percent_total_Jun_2019', 'Affordable_total_Sep_2019', 'Percent_total_Sep_2019', 'Affordable_total_Dec_2019', 'Percent_total_Dec_2019', 'Affordable_total_Mar_2020', 'Percent_total_Mar_2020', 'Affordable_total_Jun_2020', 'Percent_total_Jun_2020', 'Affordable_total_Sep_2020', 'Percent_total_Sep_2020', 'Affordable_total_Dec_2020', 'Percent_total_Dec_2020', 'Affordable_total_Mar_2021', 'Percent_total_Mar_2021', 'Affordable_total_Jun_2021', 'Percent_total_Jun_2021', 'Affordable_total_Sep_2021', 'Percent_total_Sep_2021']

In [ ]:
#save external data
path = os.getcwd().replace("notebooks","") + "data/curated/"
external.to_csv(path+'external.csv',index=False)

#### Preprocess for GNR external dataset

In [ ]:
# read GNR shapefile
GNR = gpd.read_file('../data/raw/shapefiles/gda2020_vicgrid/esrishape/whole_of_dataset/victoria/VMFOI/GNR.shp')
display(GNR.head(5))
GNR = GNR.drop(columns=['UFI', 'NAME_ID', 'VICMAP_ID', 'FEATUREC', 'NAMESTATC', 'NAMESTAT','REG_DATE','CRDATE_UFI'])
display(GNR.head(5))
path = os.getcwd().replace("notebooks","") + "data/raw/"
GNR.to_csv(path+"GNR.csv",index=False)

In [ ]:
# read GNR csv
gnr = pd.read_csv('../data/raw/GNR.csv')

# extract each useful feature (59 needed features)
fire_st = gnr[gnr['FEATURE']=='FIRE STATION']
sec_sch = gnr[gnr['FEATURE']=='SECONDARY SCHOOL']
PRIMARY_SCHOOL = gnr[gnr['FEATURE']=='PRIMARY SCHOOL']
SHOPPING_CENTRE = gnr[gnr['FEATURE']=='SHOPPING CENTRE']
PARK = gnr[gnr['FEATURE']=='PARK']
KINDERGARTEN = gnr[gnr['FEATURE']=='KINDERGARTEN']
POST_OFFICE = gnr[gnr['FEATURE']=='POST OFFICE']
SPECIAL_SCHOOL = gnr[gnr['FEATURE']=='SPECIAL SCHOOL']
PRIMARY_AND_SECONDARY_SCHOOL = gnr[gnr['FEATURE']=='PRIMARY AND SECONDARY SCHOOL']
CHILD_CARE= gnr[gnr['FEATURE']=='CHILD CARE']
RAILWAY= gnr[gnr['FEATURE']=='RAILWAY']
LOCAL_GOVERNMENT_AREA = gnr[gnr['FEATURE']=='LOCAL GOVERNMENT AREA']
SPORTS_COMPLEX = gnr[gnr['FEATURE']=='SPORTS COMPLEX']
EDUCATION_COMPLEX = gnr[gnr['FEATURE']=='EDUCATION COMPLEX']
CEMETERY = gnr[gnr['FEATURE']=='CEMETERY']
LIBRARY = gnr[gnr['FEATURE']=='LIBRARY']
UNIVERSITY = gnr[gnr['FEATURE']=='UNIVERSITY']
GENERAL_HOSPITAL = gnr[gnr['FEATURE']=='GENERAL HOSPITAL']
POLICE_STATION = gnr[gnr['FEATURE']=='POLICE STATION']
FURTHER_EDUCATION = gnr[gnr['FEATURE']=='FURTHER EDUCATION']
PLAYGROUND = gnr[gnr['FEATURE']=='PLAYGROUND']
PLANTATION = gnr[gnr['FEATURE']=='PLANTATION']
WINERY = gnr[gnr['FEATURE']=='WINERY']
BAR = gnr[gnr['FEATURE']=='BAR']
FARM = gnr[gnr['FEATURE']=='FARM']
VINEYARD= gnr[gnr['FEATURE']=='VINEYARD']
TENNIS_COURT= gnr[gnr['FEATURE']=='TENNIS COURT']
BOWLING_GREEN= gnr[gnr['FEATURE']=='BOWLING GREEN']
BAY= gnr[gnr['FEATURE']=='BAY']
PIER= gnr[gnr['FEATURE']=='PIER']
SWIMMING_POOL= gnr[gnr['FEATURE']=='SWIMMING POOL']
ART_GALLERY= gnr[gnr['FEATURE']=='ART GALLERY']
HARBOUR= gnr[gnr['FEATURE']=='HARBOUR']
WHARF= gnr[gnr['FEATURE']=='WHARF']
COMMUNITY_HEALTH_CENTRE = gnr[gnr['FEATURE']=='COMMUNITY HEALTH CENTRE']
MATERNAL_AND_CHILD_HEALTH_CENTRE= gnr[gnr['FEATURE']=='MATERNAL AND CHILD HEALTH CENTRE']
MARKET= gnr[gnr['FEATURE']=='MARKET']
PRISON= gnr[gnr['FEATURE']=='PRISON']
BOTANIC_GARDENS= gnr[gnr['FEATURE']=='BOTANIC GARDENS']
SPECIALISED_HOSPITAL= gnr[gnr['FEATURE']=='SPECIALISED HOSPITAL']
BANK= gnr[gnr['FEATURE']=='BANK']
TERTIARY_INSTITUTION= gnr[gnr['FEATURE']=='TERTIARY INSTITUTION']
AGED_CARE= gnr[gnr['FEATURE']=='AGED CARE']
SURFING_SPOT= gnr[gnr['FEATURE']=='SURFING SPOT']
AMBULANCE_STATION= gnr[gnr['FEATURE']=='AMBULANCE STATION']
COAST= gnr[gnr['FEATURE']=='COAST']
ARBORETUM= gnr[gnr['FEATURE']=='ARBORETUM']
VELODROME= gnr[gnr['FEATURE']=='VELODROME']
GOLF_COURSE= gnr[gnr['FEATURE']=='GOLF COURSE']
EQUESTRIAN_CENTRE  = gnr[gnr['FEATURE']=='EQUESTRIAN CENTRE']        
LIGHT_RAIL_STATION= gnr[gnr['FEATURE']=='LIGHT RAIL STATION']
HELIPORT= gnr[gnr['FEATURE']=='HELIPORT']
AIRPORT= gnr[gnr['FEATURE']=='AIRPORT']
FIRING_RANGE= gnr[gnr['FEATURE']=='FIRING RANGE']
IRON_ORE_PROCESSOR= gnr[gnr['FEATURE']=='IRON ORE PROCESSOR']
MILL_TIMBER_OPERATIONS= gnr[gnr['FEATURE']=='MILL/TIMBER OPERATIONS']
FERRY_STATION= gnr[gnr['FEATURE']=='FERRY STATION']
BEACH= gnr[gnr['FEATURE']=='BEACH']
MARINA = gnr[gnr['FEATURE']=='MARINA']

In [ ]:
# create a new list to put all extrated features
d_lst=[fire_st,sec_sch, PRIMARY_AND_SECONDARY_SCHOOL, PRIMARY_SCHOOL, SHOPPING_CENTRE, PARK, KINDERGARTEN, POST_OFFICE, SPECIAL_SCHOOL, CHILD_CARE, RAILWAY,
BAR, LOCAL_GOVERNMENT_AREA, SPORTS_COMPLEX, EDUCATION_COMPLEX, CEMETERY, LIBRARY, UNIVERSITY, GENERAL_HOSPITAL, POLICE_STATION, FURTHER_EDUCATION, PLAYGROUND, 
PLANTATION, WINERY, BAR, FARM, VINEYARD, TENNIS_COURT, BOWLING_GREEN, BAY, PIER, SWIMMING_POOL, ART_GALLERY, HARBOUR, WHARF, COMMUNITY_HEALTH_CENTRE,
MATERNAL_AND_CHILD_HEALTH_CENTRE, MARKET, PRISON, BOTANIC_GARDENS, SPECIALISED_HOSPITAL, BANK, TERTIARY_INSTITUTION, AGED_CARE, SURFING_SPOT,
AMBULANCE_STATION, COAST, ARBORETUM, VELODROME, GOLF_COURSE, EQUESTRIAN_CENTRE, LIGHT_RAIL_STATION, HELIPORT, AIRPORT, FIRING_RANGE, IRON_ORE_PROCESSOR,
MILL_TIMBER_OPERATIONS, FERRY_STATION, BEACH, MARINA]

# combine all the features into one dataframe and remove the useless colunms
clean_GNR = pd.concat(d_lst)

# replace marina(4) and wharf(11) into harbour(18 +4+11)
clean_GNR = clean_GNR.replace('MARINA', 'HARBOUR')
clean_GNR = clean_GNR.replace('WHARF', 'HARBOUR')

clean_GNR

In [ ]:
#save GNR Data
path = os.getcwd().replace("notebooks","") + "data/curated/"
clean_GNR.to_csv(path+'GNR.csv',index=False)